<br><br><b><font size=10> CONTINUOUS CONTROL</font></b>
#### <i>...implementation for Udacity Deep Reinforcement Learning 
<hr><hr><hr>

## Initialize Imports for the notebook
This Notebook uses code from separate python files where most of the implementation is handled

In [1]:
import env as E
from buffers import ReplayBuffer, nStepBuffer
from agent import D4PG_Agent
import models

# from agent import Agent
#from get_args import get_args

import os.path
import time
import re
import sys

import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T

In [2]:
import multiprocessing as multi
multi.cpu_count()

16

## Manually declare an ARGS class
<i> For testing code in the notebook, to take the place of argparser in the command line.

In [3]:
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        
args = Args()

<b>Confirm that the args are all set the way we want them.

In [4]:
for arg in vars(args):
    if arg == "sep": continue
    print("{}: {}".format(arg.upper(), getattr(args, arg)))

TRAIN: True
NOGRAPHICS: False
NUM_EPS: 10
ROLLOUT: 5
BATCHSIZE: 64
PRETRAIN: 1000


## Load the environment
<i> & print a bit of information contained in the wrapper class

In [5]:
env = E.Environment(args)
print("State size:", env.state_size)
print("Action size:", env.action_size)
print("Num Agents:", env.agent_count)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State size: 33
Action size: 4
Num Agents: 20


<hr>

# Test code as it's developed

<hr>

## Take random actions in the environment below 
<i>
-to check that code is working<br>
-to get familiar with the environment

In [7]:
env.train = False
env.reset()

In [8]:
scores = np.zeros(env.agent_count)
states = env.states
for i in range(20):
    actions = np.random.randn(env.agent_count, env.action_size)
    actions = np.clip(actions, -1, 1)
    rewards, dones = env.step(actions)
    scores += rewards
    states = env.states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

Total score (averaged over agents) this episode: 0.0


### Force-reload modules as they're updated
<i> This notebook was developed as the code is written in Atom, the below cell reloads the modules as they're needed.

In [59]:
import importlib
import agent
import buffers
import copy
importlib.reload(agent)
importlib.reload(buffers)
importlib.reload(models)

<module 'models' from 'C:\\Users\\mattdoll\\Development\\deep-reinforcement-learning\\p2_continuous-control\\models.py'>

In [ ]:
# agent = agent.D4PG_Agent(env.state_size, env.action_size, env.agent_count, args.rollout)
# print(agent.__class__.__name__)
# print(agent.memory)
# agent.initialize_memory(10, env)

In [51]:
env.reset()
states = env.states

### Test out Actor actions without training
<i> Test the <b>Actor</b> network

In [31]:
actor = models.ActorNet(env.state_size, env.action_size)

In [26]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(10):
    actions = actor(torch.from_numpy(states).float()).detach().numpy()
    print("ACTIONS:", actions)
    rewards, dones = env.step(actions)
    scores += rewards
    states = env.states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

ACTIONS: [[-1.7259008e-06  2.9305425e-03  1.0615054e-03 -1.2740623e-03]
 [-6.4743654e-06  2.9114615e-03  1.0617129e-03 -1.2663912e-03]
 [-9.1323964e-06  2.9120664e-03  1.0722961e-03 -1.2598998e-03]
 [-2.2053719e-06  2.9295518e-03  1.0642209e-03 -1.2742883e-03]
 [-1.0844742e-05  2.9116794e-03  1.0721537e-03 -1.2589438e-03]
 [-1.7902494e-08  2.9215384e-03  1.0735834e-03 -1.2690779e-03]
 [-1.3820769e-05  2.9063930e-03  1.0675727e-03 -1.2610800e-03]
 [-9.1163820e-06  2.9099404e-03  1.0617779e-03 -1.2654751e-03]
 [-9.9992176e-06  2.9077125e-03  1.0629190e-03 -1.2643565e-03]
 [-2.1913875e-06  2.9234043e-03  1.0620687e-03 -1.2720514e-03]
 [-6.8611989e-06  2.9124150e-03  1.0616282e-03 -1.2666581e-03]
 [-2.9900548e-06  2.9294705e-03  1.0640930e-03 -1.2736254e-03]
 [-1.2416727e-05  2.9059744e-03  1.0660122e-03 -1.2629260e-03]
 [-3.3348115e-06  2.9297816e-03  1.0609163e-03 -1.2737125e-03]
 [-6.0319144e-07  2.9288740e-03  1.0653370e-03 -1.2750762e-03]
 [-7.7205405e-06  2.9101579e-03  1.0619377e-03

 [-9.52018308e-06  2.91279750e-03  1.07282645e-03 -1.25881517e-03]]
ACTIONS: [[-1.2740202e-06  2.9321588e-03  1.0618506e-03 -1.2753253e-03]
 [-3.8900253e-06  2.9149640e-03  1.0617253e-03 -1.2676592e-03]
 [-9.1611946e-06  2.9120285e-03  1.0722891e-03 -1.2598580e-03]
 [-2.7170900e-06  2.9312004e-03  1.0628398e-03 -1.2743858e-03]
 [-8.8375382e-06  2.9128431e-03  1.0728597e-03 -1.2591853e-03]
 [-4.4632179e-07  2.9198339e-03  1.0742212e-03 -1.2678656e-03]
 [-1.3721634e-05  2.9095325e-03  1.0709692e-03 -1.2579075e-03]
 [-1.1522832e-05  2.9072564e-03  1.0636416e-03 -1.2636796e-03]
 [-1.0107535e-05  2.9076384e-03  1.0630428e-03 -1.2642698e-03]
 [-2.6952330e-06  2.9265569e-03  1.0618852e-03 -1.2731770e-03]
 [-6.4652049e-06  2.9130832e-03  1.0615839e-03 -1.2669273e-03]
 [-3.5705016e-06  2.9321718e-03  1.0616485e-03 -1.2740555e-03]
 [-1.2580669e-05  2.9058962e-03  1.0663434e-03 -1.2627098e-03]
 [-2.9187795e-06  2.9310251e-03  1.0612757e-03 -1.2738280e-03]
 [-4.0938903e-07  2.9264283e-03  1.067730

### Test out Critic scores without training
<i> Test the <b>Critic</b> network

In [60]:
critic = models.CriticNet(env.state_size, env.action_size)

In [ ]:
scores = np.zeros(env.agent_count)
states = env.states
for i in range(10):
    states = torch.from_numpy(states).float()
    actions = actor(states).detach().numpy()
    rewards, dones = env.step(actions)
    scores += rewards
    values = critic(states, torch.from_numpy(actions))
    #print(critic.Q_val)
    #print(values.sample())
    states = env.states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

In [61]:
actions = np.random.randn(env.agent_count, env.action_size)
actions = np.clip(actions, -1, 1).astype(np.float32)
states = torch.from_numpy(env.states).float()

In [62]:
probs, log_probs = critic(states, torch.from_numpy(actions))

In [64]:
print(probs.shape)
print(log_probs.shape)

torch.Size([20, 51])
torch.Size([20, 51])


In [ ]:
env.close()